# Examples

## Private LLM calls with santized prompts

In [10]:
import requests, json
api_url = "http://localhost:8000"

request = {
    "provider": "openai",
    "model": "gpt-3.5-turbo",
    "messages": [
        {
            "role": "system",
            "content": "Context: Rushabh is very passionate about technology and loves tinkering with new products. He enjoys spending quality time in nature in his free time. He's early in his career, so he's willing to forgo salary for career growth. Jonah is an entrepreneur at heart and believes that tech companies are the best suited for making lots of money. He's willing to work long hours, as long as he's working on a cause he believes in.",
            "name": "context"
        },
        {
            "role": "system",
            "content": "Please provide a JSON response with the following format: {name: [ordered list of features]}"
        },
        {
            "role": "user",
            "content": "Rank the importance of these features of jobs for Rushabh and Jonah: 1. Compensation 2. Work Life Balance 3. Company Mission",
            "name": "rankings"
        }
    ]
}

res = requests.post(api_url + "/llm/chat/completions?debug=true", json=request)
response = res.json()



####  What AI Sees

In [13]:
print(json.dumps(response["raw_request"], indent=4))

[
    {
        "role": "system",
        "content": "Ensure that all placeholders, including those inside quotes, are enclosed bythe greek letter alpha (\u03b1), exactly as I have done in this prompt. You MUST use the greek letter (\u03b1) to indicate placeholders. Use the context below to help answer the question and follow the desired format accurately.",
        "name": null,
        "function": null
    },
    {
        "role": "system",
        "content": "Context: \u03b17xKlCkUUaKk9Qz4GnYfAOLbWGkMHuwU6XaQDG35E10w=\u03b1 is very passionate about technology and loves tinkering with new products. He enjoys spending quality time in nature in his free time. He's early in his career, so he's willing to forgo salary for career growth. \u03b1YjlT/8nMyx7fvAOozx/Ht6Xak00k4uAHOBFi8g+/80Y=\u03b1 is an entrepreneur at heart and believes that tech companies are the best suited for making lots of money. He's willing to work long hours, as long as he's working on a cause he believes in.",
     

####  What users see

In [11]:
print(json.dumps(response["messages"], indent=4))

[
    {
        "role": "assistant",
        "content": "{\"Rushabh\": [\"Company Mission\", \"Work Life Balance\", \"Compensation\"], \"Jonah\": [\"Compensation\", \"Company Mission\", \"Work Life Balance\"]}",
        "name": null,
        "function": null
    }
]


## Secure RAG with Santized Documents and Prompts



### Set Up Vector DB
We assume you are using postgres with PGVector. Please email sumant@pontus.so for any support for other vector databases.

In [14]:
documents_from_wiki = ["Bill Burr", "Marie Kondo", "Garry Kasparov"]
requests.post(api_url + "/demo/load_wiki_pages", json={"page_names": documents_from_wiki})

<Response [200]>

### Call LLM
To see what prompts are sent to OpenAI, please pass the query param debug as true (?debug=true) and to enable RAG (?rag=true)

In [26]:
request = {
	"provider": "openai",
	"model": "gpt-3.5-turbo",
	"messages": [

		{
			"role": "system",
			"content": "Please provide a JSON response with the following format: {'Cards against Humanity': [ordered list of Bill, Garry, or Marie], 'Go': [ordered list of Bill, Garry, or Marie], 'Solitare': [ordered list of Bill, Garry, or Marie]}"
		},
		{
			"role": "user",
			"content": "For each of the games 'Cards against Humanity', 'Go', and 'Solitaire', please order Marie Kondo, Garry Kasparov, and Bill Burr based on how much they 'd enjoy playing each game."
		}
	],
	"titles": ["Bill Burr", "Marie Kondo", "Garry Kasparov"]
}


res_with_rag = requests.post(api_url + "/llm/chat/completions?debug=true&enable_rag=true", json=request)
response_with_rag = res_with_rag.json()


#### What AI sees

In [27]:
print(json.dumps(response_with_rag["raw_request"], indent=4))

[
    {
        "role": "system",
        "content": "Ensure that all placeholders, including those inside quotes, are enclosed bythe greek letter alpha (\u03b1), exactly as I have done in this prompt. You MUST use the greek letter (\u03b1) to indicate placeholders. Use the context below to help answer the question and follow the desired format accurately.",
        "name": null,
        "function": null
    },
    {
        "role": "system",
        "content": "Please provide a JSON response with the following format: {'Cards against Humanity': [ordered list of \u03b1sbAy+9LFnKdGZXK6LSah07/LlYTyuL46U3eJzsAc3Xc=\u03b1, \u03b1plHj06iYn1Q6cevi00kd2tbV4xNI3U8IJGgCTs+W/mM=\u03b1, or \u03b1VdNO1B3qV4EzUc5fQLlnUXwafYAlSTyq8KxqiAn3fkQ=\u03b1], 'Go': [ordered list of \u03b1sbAy+9LFnKdGZXK6LSah07/LlYTyuL46U3eJzsAc3Xc=\u03b1, \u03b1plHj06iYn1Q6cevi00kd2tbV4xNI3U8IJGgCTs+W/mM=\u03b1, or \u03b1VdNO1B3qV4EzUc5fQLlnUXwafYAlSTyq8KxqiAn3fkQ=\u03b1], 'Solitare': [ordered list of \u03b1sbAy+9LFnKdGZXK6L

#### What User See

In [28]:
print(json.dumps(response_with_rag["messages"], indent=4))

[
    {
        "role": "assistant",
        "content": "{'Cards against Humanity': ['Garry Kasparov', 'Bill Burr', 'Marie Kondo'], 'Go': ['Garry Kasparov', 'Marie Kondo', 'Bill Burr'], 'Solitare': ['Marie Kondo', 'Garry Kasparov', 'Bill Burr']}",
        "name": null,
        "function": null
    }
]
